<h2>Parte 1</h2>

In [6]:
juros_selic = spark.read.json("/user/rodrigo/data/juros_selic/juros_selic.json")
juros_selic.show(5)

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1.27|
|01/07/1986| 1.95|
|01/08/1986| 2.57|
|01/09/1986| 2.94|
|01/10/1986| 1.96|
+----------+-----+
only showing top 5 rows



In [7]:
from pyspark.sql.functions import unix_timestamp,from_unixtime, col, substring, split

convUnix = juros_selic.withColumn("timestamp", unix_timestamp(col("data"), "dd/MM/yyyy"))
convUnix.show(5)

+----------+-----+---------+
|      data|valor|timestamp|
+----------+-----+---------+
|01/06/1986| 1.27|517968000|
|01/07/1986| 1.95|520560000|
|01/08/1986| 2.57|523238400|
|01/09/1986| 2.94|525916800|
|01/10/1986| 1.96|528508800|
+----------+-----+---------+
only showing top 5 rows



In [8]:
convUnixData = convUnix.withColumn("new data", from_unixtime("timestamp", "MM/dd/yyyy"))
convUnixData.show(5)

+----------+-----+---------+----------+
|      data|valor|timestamp|  new data|
+----------+-----+---------+----------+
|01/06/1986| 1.27|517968000|06/01/1986|
|01/07/1986| 1.95|520560000|07/01/1986|
|01/08/1986| 2.57|523238400|08/01/1986|
|01/09/1986| 2.94|525916800|09/01/1986|
|01/10/1986| 1.96|528508800|10/01/1986|
+----------+-----+---------+----------+
only showing top 5 rows



In [9]:
convUnixDataAno = convUnix.withColumn("ano_unix", from_unixtime("timestamp", "yyyy"))
convUnixDataAno.show(5)

+----------+-----+---------+--------+
|      data|valor|timestamp|ano_unix|
+----------+-----+---------+--------+
|01/06/1986| 1.27|517968000|    1986|
|01/07/1986| 1.95|520560000|    1986|
|01/08/1986| 2.57|523238400|    1986|
|01/09/1986| 2.94|525916800|    1986|
|01/10/1986| 1.96|528508800|    1986|
+----------+-----+---------+--------+
only showing top 5 rows



In [10]:
convUnixDataAnoStr = convUnixDataAno.withColumn("ano_str", substring(col("data"),7,9))
convUnixDataAnoStr.show(5)

+----------+-----+---------+--------+-------+
|      data|valor|timestamp|ano_unix|ano_str|
+----------+-----+---------+--------+-------+
|01/06/1986| 1.27|517968000|    1986|   1986|
|01/07/1986| 1.95|520560000|    1986|   1986|
|01/08/1986| 2.57|523238400|    1986|   1986|
|01/09/1986| 2.94|525916800|    1986|   1986|
|01/10/1986| 1.96|528508800|    1986|   1986|
+----------+-----+---------+--------+-------+
only showing top 5 rows



In [11]:
convUnixDataAnoStrSplit = convUnixDataAnoStr.withColumn("ano_str2", split(col("data"),"/").getItem(2))
convUnixDataAnoStrSplit.show(5)

+----------+-----+---------+--------+-------+--------+
|      data|valor|timestamp|ano_unix|ano_str|ano_str2|
+----------+-----+---------+--------+-------+--------+
|01/06/1986| 1.27|517968000|    1986|   1986|    1986|
|01/07/1986| 1.95|520560000|    1986|   1986|    1986|
|01/08/1986| 2.57|523238400|    1986|   1986|    1986|
|01/09/1986| 2.94|525916800|    1986|   1986|    1986|
|01/10/1986| 1.96|528508800|    1986|   1986|    1986|
+----------+-----+---------+--------+-------+--------+
only showing top 5 rows



In [12]:
convUnixDataAnoStrSplit.write.csv("/user/rodrigo/juros_selic_americano")

In [13]:
!hdfs dfs -ls /user/rodrigo/juros_selic_americano

Found 2 items
-rw-r--r--   2 root supergroup          0 2021-10-31 23:16 /user/rodrigo/juros_selic_americano/_SUCCESS
-rw-r--r--   2 root supergroup      16401 2021-10-31 23:16 /user/rodrigo/juros_selic_americano/part-00000-c444fd01-d146-4daf-a099-a3b89d6f4d87-c000.csv


<h2>Parte 2</h2>

In [14]:
juros_selic = spark.read.json("hdfs://namenode:8020/user/rodrigo/data/juros_selic/juros_selic.json")
juros_selic.show(5)

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1.27|
|01/07/1986| 1.95|
|01/08/1986| 2.57|
|01/09/1986| 2.94|
|01/10/1986| 1.96|
+----------+-----+
only showing top 5 rows



In [15]:
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import sum, avg, min, max, count, desc

In [16]:
convUnixDataAno = juros_selic.withColumn("ano_str", substring(col("data"),7,9))
convUnixDataAno = convUnixDataAno.withColumn("valor_float",col("valor").cast(DoubleType()))

convUnixDataAno.show(5)

+----------+-----+-------+-----------+
|      data|valor|ano_str|valor_float|
+----------+-----+-------+-----------+
|01/06/1986| 1.27|   1986|       1.27|
|01/07/1986| 1.95|   1986|       1.95|
|01/08/1986| 2.57|   1986|       2.57|
|01/09/1986| 2.94|   1986|       2.94|
|01/10/1986| 1.96|   1986|       1.96|
+----------+-----+-------+-----------+
only showing top 5 rows



In [17]:
convUnixDataAnoGroup = convUnixDataAno.groupBy("ano_str")\
    .agg(
        count("data").alias("meses"),
        avg("valor_float").alias("valor_medio"),
        min("valor_float").alias("valor_minimo"),
        max("valor_float").alias("valor_maximo"),
    ).sort(desc("ano_str"))
convUnixDataAnoGroup.show()

+-------+-----+------------------+------------+------------+
|ano_str|meses|       valor_medio|valor_minimo|valor_maximo|
+-------+-----+------------------+------------+------------+
|   2019|    2|             0.455|        0.37|        0.54|
|   2018|   12|              0.52|        0.47|        0.58|
|   2017|   12|0.7941666666666665|        0.54|        1.09|
|   2016|   12|1.1000000000000003|         1.0|        1.22|
|   2015|   12|             1.045|        0.82|        1.18|
|   2014|   12|0.8666666666666666|        0.77|        0.96|
|   2013|   12|              0.66|        0.49|        0.81|
|   2012|   12|0.6808333333333333|        0.54|        0.89|
|   2011|   12|              0.92|        0.84|        1.07|
|   2010|   12|0.7808333333333333|        0.59|        0.93|
|   2009|   12|0.7916666666666666|        0.66|        1.05|
|   2008|   12|             0.985|         0.8|        1.18|
|   2007|   12|            0.9375|         0.8|        1.08|
|   2006|   12|         

In [18]:
convUnixDataAnoGroup.write.orc("/user/paulo/relatorioAnual", compression="zlib")

In [19]:
!hdfs dfs -ls /user/paulo/relatorioAnual

Found 35 items
-rw-r--r--   2 root supergroup          0 2021-10-31 23:16 /user/paulo/relatorioAnual/_SUCCESS
-rw-r--r--   2 root supergroup        610 2021-10-31 23:16 /user/paulo/relatorioAnual/part-00000-53802505-a674-4a89-8827-1225b39a4470-c000.zlib.orc
-rw-r--r--   2 root supergroup        614 2021-10-31 23:16 /user/paulo/relatorioAnual/part-00001-53802505-a674-4a89-8827-1225b39a4470-c000.zlib.orc
-rw-r--r--   2 root supergroup        614 2021-10-31 23:16 /user/paulo/relatorioAnual/part-00002-53802505-a674-4a89-8827-1225b39a4470-c000.zlib.orc
-rw-r--r--   2 root supergroup        602 2021-10-31 23:16 /user/paulo/relatorioAnual/part-00003-53802505-a674-4a89-8827-1225b39a4470-c000.zlib.orc
-rw-r--r--   2 root supergroup        614 2021-10-31 23:16 /user/paulo/relatorioAnual/part-00004-53802505-a674-4a89-8827-1225b39a4470-c000.zlib.orc
-rw-r--r--   2 root supergroup        604 2021-10-31 23:16 /user/paulo/relatorioAnual/part-00005-53802505-a674-4a89-8827-1225b39a4470-c000.zlib.orc
-r